# Pandas és Polars mérközések
![Pandas vs Polars](assets/polars_pandas_arm_wrestling.png)



In [2]:
import pandas as pd
import polars as pl
import numpy as np

In [6]:
import time

verbose = True
#region-> Példa Polars és Pandas DataFrame létrehozására, feltöltése
number_of_rows = 10_000_000
max_fill_value = 110
exclude_threshold = 100

values_x = np.random.randint(1, max_fill_value, number_of_rows)
values_y = np.random.randint(1, max_fill_value, number_of_rows)
values_z = np.random.randint(1, max_fill_value, number_of_rows)
group_values = np.random.choice(list("ABC"), number_of_rows)

df_pd = pd.DataFrame({
    "x": values_x,
    "y": values_y,
    "z": values_z,
    "group": group_values
})
df_pl = pl.from_pandas(df_pd)
print(df_pl.head(10))
#endregion

#region -> Műveletek Polars DataFrame-nél (szűrés, csoportosítás, aggregálás, rendezés)
start_time = time.time()
result_pl = (
    df_pl
    .filter(pl.col("x") > exclude_threshold)  # Filter rows where x > 100
    .group_by("group")
    .agg([
        pl.col("x").mean().alias("x_mean"),
        pl.col("y").mean().alias("y_mean"),
        pl.col("z").mean().alias("z_mean")
    ])
    .sort('group')
)
polars_time = time.time() - start_time
#enregion

if verbose:
    print(f"Az eredmény Polars dataframe esetén {polars_time:.4f} másodperc alatt készült.")
    print(result_pl)

#region -> Műveletek Pandas DataFrame-nél (szűrés, csoportosítás, aggregálás, rendezés)
start_time = time.time()
result_pd = (
    df_pd[df_pd["x"] > exclude_threshold]
    .groupby("group")[["x", "y", "z"]]
    .mean()
    .rename(columns={"x": "x_mean", "y": "y_mean", "z": "z_mean"})
    .reset_index()
    .sort_values(by="group")
)
pandas_time = time.time() - start_time
#endregion


if verbose:
    print(f"Az eredmény Polars dataframe esetén {pandas_time:.4f} másodperc alatt készült.")
    print(result_pl)

df_time_results = pl.DataFrame({
    "library": ["Polars", "Pandas"],
    "time": [polars_time, pandas_time]
})

df_time_results \
    .plot \
    .bar(x="library", y="time") \
    .properties(title="Mérési eredmények", width=400, height=300)


shape: (10, 4)
┌─────┬─────┬─────┬───────┐
│ x   ┆ y   ┆ z   ┆ group │
│ --- ┆ --- ┆ --- ┆ ---   │
│ i64 ┆ i64 ┆ i64 ┆ str   │
╞═════╪═════╪═════╪═══════╡
│ 58  ┆ 73  ┆ 46  ┆ C     │
│ 48  ┆ 48  ┆ 61  ┆ A     │
│ 98  ┆ 84  ┆ 38  ┆ C     │
│ 57  ┆ 101 ┆ 19  ┆ C     │
│ 17  ┆ 91  ┆ 20  ┆ A     │
│ 74  ┆ 76  ┆ 83  ┆ A     │
│ 46  ┆ 106 ┆ 97  ┆ C     │
│ 91  ┆ 8   ┆ 64  ┆ B     │
│ 23  ┆ 102 ┆ 80  ┆ B     │
│ 69  ┆ 106 ┆ 22  ┆ C     │
└─────┴─────┴─────┴───────┘
Az eredmény Polars dataframe esetén 0.0327 másodperc alatt készült.
shape: (3, 4)
┌───────┬────────────┬───────────┬───────────┐
│ group ┆ x_mean     ┆ y_mean    ┆ z_mean    │
│ ---   ┆ ---        ┆ ---       ┆ ---       │
│ str   ┆ f64        ┆ f64       ┆ f64       │
╞═══════╪════════════╪═══════════╪═══════════╡
│ A     ┆ 105.000731 ┆ 54.931733 ┆ 54.97635  │
│ B     ┆ 105.00315  ┆ 55.036729 ┆ 54.978507 │
│ C     ┆ 104.994011 ┆ 55.025019 ┆ 55.03172  │
└───────┴────────────┴───────────┴───────────┘
Az eredmény Polars dataframe ese

alt.Chart(...)